In [ ]:
from peft import PeftModel
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [9]:
import os
from google.colab import drive

# Подключаем Google Drive для сохранения результатов
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
def test_model(model, prompt):
    # Форматируем как при обучении
    formatted_prompt = f'''
You are a PascalABC.NET coding assistant. Follow these rules:
1. Write only PascalABC.NET code
2. Provide complete programs when possible
3. Use modern PascalABC.NET features

### Instruction:
{prompt}
### Response:
'''

    inputs = tokenizer(formatted_prompt, return_tensors="pt", max_length=512, truncation=True).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1024,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

def test_model_raw(prompt):
  formatted_prompt = f'''
    ### Instruction:
    {prompt}
    ### Response:
    '''

  inputs = tokenizer(formatted_prompt, return_tensors="pt", max_length=512, truncation=True).to(model2.device)

  print(f"Inputs device: {inputs['input_ids'].device}")

  with torch.no_grad():
      outputs = model2.generate(
          **inputs,
          max_new_tokens=1024,
          temperature=0.7,
          do_sample=True,
          pad_token_id=tokenizer.eos_token_id
      )

  response = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return response

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True, torch_dtype=torch.bfloat16).cuda()
messages=[
    { 'role': 'user', 'content': "write quick sort algorithm."}
]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
# tokenizer.eos_token_id is the id of <|EOT|> token
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Sure, here is a simple implementation of the Quick Sort algorithm in Python:

```python
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quick_sort(left) + middle + quick_sort(right)

# Test the function
print(quick_sort([3,6,8,10,1,2,1]))
# Output: [1, 1, 2, 3, 6, 8, 10]
```

This algorithm works by selecting a 'pivot' element from the array and partitioning the other elements into two sub-arrays, according to whether they are less than or greater than the pivot. The sub-arrays are then recursively sorted.

The pivot selection is done in a way that it divides the array into two halves, one with elements less than the pivot and one with elements greater than the pivot. This is done to ensure that the pivot is in its final sorted position.

The time complexity of Quick Sort is O(n log n) in all cases (worst,

In [ ]:
print("Model structure:")
print(model)

Model structure:
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 2048)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (down_proj): Linear(in_features=5504, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-06)
      )
    )
    (norm): LlamaRMSNorm((2048,), 

In [ ]:
!pip install jsonlines

In [ ]:
print(f"Number of transformer layers: {len(model.model.layers)}")

for i, layer in enumerate(model.model.layers):
    print(f"\n--- Layer {i} ---")
    for name, param in layer.named_parameters():
        print(f"  {name}: {param.shape}")

Number of transformer layers: 24

--- Layer 0 ---
  self_attn.q_proj.weight: torch.Size([2048, 2048])
  self_attn.k_proj.weight: torch.Size([2048, 2048])
  self_attn.v_proj.weight: torch.Size([2048, 2048])
  self_attn.o_proj.weight: torch.Size([2048, 2048])
  mlp.gate_proj.weight: torch.Size([5504, 2048])
  mlp.up_proj.weight: torch.Size([5504, 2048])
  mlp.down_proj.weight: torch.Size([2048, 5504])
  input_layernorm.weight: torch.Size([2048])
  post_attention_layernorm.weight: torch.Size([2048])

--- Layer 1 ---
  self_attn.q_proj.weight: torch.Size([2048, 2048])
  self_attn.k_proj.weight: torch.Size([2048, 2048])
  self_attn.v_proj.weight: torch.Size([2048, 2048])
  self_attn.o_proj.weight: torch.Size([2048, 2048])
  mlp.gate_proj.weight: torch.Size([5504, 2048])
  mlp.up_proj.weight: torch.Size([5504, 2048])
  mlp.down_proj.weight: torch.Size([2048, 5504])
  input_layernorm.weight: torch.Size([2048])
  post_attention_layernorm.weight: torch.Size([2048])

--- Layer 2 ---
  self_attn.

In [ ]:
messages=[
    { 'role': 'user', 'content': "write a Fibonacci numbers printing function in Pascal using modern Pascal style"}
]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
# tokenizer.eos_token_id is the id of <|EOT|> token
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Here is a simple implementation of a Fibonacci numbers printing function in Pascal using the modern style:

```pascal
program Fibonacci;

function FibonacciNumbers(n: integer): string;
var
  a, b, c: integer;
begin
  if n <= 0 then
    result := ''
  else if n = 1 then
    result := '0'
  else if n = 2 then
    result := '0, 1'
  else
  begin
    a := 0;
    b := 1;
    c := 0;
    result := '0, 1';
    while c < n - 2 do
    begin
      c := a + b;
      a := b;
      b := c;
      result := result + ', ' + IntToStr(c);
    end;
  end;
end;

begin
  writeln(FibonacciNumbers(10));
end.
```

This program defines a function `FibonacciNumbers` that generates the first `n` Fibonacci numbers. It starts with the first two Fibonacci numbers (0 and 1) and then continuously adds the last two numbers to generate the next Fibonacci number. The function returns a string of comma-separated Fibonacci numbers.

The `begin` statement at the end of the program calls `FibonacciNumbers` with `n` set to 1

In [ ]:
messages=[
    { 'role': 'user', 'content': "напиши на Паскале функцию быстрой сортировки. Все переменные называй с большой буквы."}
]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
# tokenizer.eos_token_id is the id of <|EOT|> token
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


В Паскале нет встроенной функции для быстрой сортировки, но есть встроенные функции для сортировки списка. Вот примеры:

1. Выбор сортировки:

```pascal
procedure SelectionSort(var A: array of Integer; L, R: Integer);
var
  i, j, max_i, temp: Integer;
begin
  for i := L to R - 1 do
  begin
    max_i := i;
    for j := i + 1 to R do
      if A[j] > A[max_i] then
        max_i := j;
    temp := A[i];
    A[i] := A[max_i];
    A[max_i] := temp;
  end;
end;
```

2. Вставка сортировки:

```pascal
procedure InsertionSort(var A: array of Integer; L, R: Integer);
var
  i, j, key: Integer;
begin
  for i := L + 1 to R do
  begin
    key := A[i];
    j := i - 1;
    while (j >= L) and (A[j] > key) do
    begin
      A[j + 1] := A[j];
      j := j - 1;
    end;
    A[j + 1] := key;
  end;
end;
```

3. Шелла сортировки:

```pascal
procedure ShellSort(var A: array of Integer; L, R: Integer);
var
  gap, i, j, temp: Integer;
begin
  gap := R div 2;
  while gap > 0 do
  begin
    for i := gap to R do
 

In [ ]:
messages=[
    { 'role': 'user', 'content': "Write Hanoi Towers algorithm in python."}
]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
# tokenizer.eos_token_id is the id of <|EOT|> token
outputs = model.generate(inputs, max_new_tokens=1024, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


The Hanoi Towers problem is a classic problem in computer science, which is a mathematical puzzle that can be solved using recursion. The problem consists of three rods and a number of disks of different sizes which can slide onto any rod. The puzzle starts with the disks in a neat stack in ascending order of size on one rod, the smallest at the top, thus making a conical shape.

The objective of the puzzle is to move the entire stack to another rod, obeying the following simple rules:

1. Only one disk can be moved at a time.
2. Each move consists of taking the upper disk from one of the stacks and placing it on top of another stack or on an empty rod.
3. No disk may be placed on top of a smaller disk.

Here is a Python implementation of the Hanoi Towers algorithm:

```python
def hanoi(n, source, target, auxiliary):
    if n > 0:
        # Move n - 1 disks from source to auxiliary, so they are out of the way
        hanoi(n - 1, source, auxiliary, target)

        # Move the nth disk 

In [ ]:
messages=[
    { 'role': 'user', 'content': "Write an algorithm which solves n qweens problem."}
]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
# tokenizer.eos_token_id is the id of <|EOT|> token
outputs = model.generate(inputs, max_new_tokens=1024, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


The n-Queens problem is a classic problem in computer science, which asks how to place n queens on an n×n chessboard so that no two queens threaten each other.

Here is a simple algorithm to solve the n-Queens problem:

1. Initialize a 2D array of size n x n, where each cell represents a row.

2. For each row, place a queen in each column.

3. Check for any two queens in the same row. If a queen is found, return false.

4. Check for any two queens in the same column. If a queen is found, return false.

5. Check for any two queens in the same diagonal. If a queen is found, return false.

6. If no conflict is found, return true.

Here is a Python implementation of the above algorithm:

```python
def solve_n_queens(n):
    def is_attack(i, j):
        # Check for any queen in the same row
        for k in range(0, n):
            if board[i][k] == 1:
                return True

        # Check for any queen in the same column
        for k in range(0, n):
            if board[k][j] == 1:

In [ ]:
messages=[
    { 'role': 'user', 'content': '''Write Pascal code in MODERN PASCAL syntax with the following requirement:
    - MUST use inline variable declaration in FOR loops: "for var i := 1 to n do"
    - Do NOT use traditional variable declaration before the loop
    - This is for Free Pascal or PascalABC.NET compiler

    Write a function that counts 'A' symbols in a string.'''}
]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
# tokenizer.eos_token_id is the id of <|EOT|> token
outputs = model.generate(inputs, max_new_tokens=1024, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Here is a simple function in Free Pascal that counts the number of 'A' symbols in a string:

```pascal
function CountA(const s: string): integer;
var
  i: integer;
begin
  Result := 0;
  for i := 1 to length(s) do
    if s[i] = 'A' then
      Inc(Result);
end;
```

This function takes a string as input and initializes a counter to 0. It then iterates over each character in the string, checking if it is 'A'. If it is, it increments the counter. Finally, it returns the counter, which represents the number of 'A' symbols in the string.

Please note that this function is case-sensitive, meaning it will count 'A' and 'a' as the same symbol. If you want to count 'A' and 'a' as different symbols, you would need to convert the string to either lower or upper case before counting.



In [ ]:
messages=[
    { 'role': 'user', 'content': '''
    Follow these rules when writing Pascal code:
    - Use "for var i := 1 to n do" pattern instead of "var i : integer; for i := 1 to n do"

    Translate this Python code to Pascal:
    def is_attack(i, j):
        # Check for any queen in the same row
        for k in range(0, n):
            if board[i][k] == 1:
                return True

        # Check for any queen in the same column
        for k in range(0, n):
            if board[k][j] == 1:
                return True

        # Check for any queen in the same diagonal
        for k in range(0, n):
            for l in range(0, n):
                if (k + l == i + j) or (k - l == i - j):
                    if board[k][l] == 1:
                        return True

        return False

    def solve_n_queens_util(n, i):
        # base case
        if i == n:
            return True

        for j in range(0, n):
            if not is_attack(i, j):
                # place queen at board[i][j]
                board[i][j] = 1

                # recur to place rest of the queens
                if solve_n_queens_util(n, i + 1) == True:
                    return True

                # If placing queen in board[i][j] doesn't lead to a solution, then
                # remove queen from board[i][j]
                board[i][j] = 0

        return False

    board = [[0]*n for _ in range(n)]

    if solve_n_queens_util(n, 0) == False:
        print("Solution does not exist")
        return False

    print_board(board, n)
    return True
    '''}
]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
# tokenizer.eos_token_id is the id of <|EOT|> token
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Here is the Pascal code equivalent to the given Python code:

```pascal
function is_attack(i, j: integer; n: integer): boolean;
var
  k: integer;
begin
  // Check for any queen in the same row
  for k := 0 to n-1 do
    if board[i][k] = 1 then
      exit(true);

  // Check for any queen in the same column
  for k := 0 to n-1 do
    if board[k][j] = 1 then
      exit(true);

  // Check for any queen in the same diagonal
  for k := 0 to n-1 do
    for l := 0 to n-1 do
      if (k + l = i + j) or (k - l = i - j) then
        if board[k][l] = 1 then
          exit(true);

  exit(false);
end;

function solve_n_queens_util(n, i: integer): boolean;
var
  j: integer;
begin
  // base case
  if i = n then
    exit(true);

  for j := 0 to n-1 do
    if not is_attack(i, j, n) then
    begin
      // place queen at board[i][j]
      board[i][j] := 1;

      // recur to place rest of the queens
      if solve_n_queens_util(n, i + 1) then
        exit(true);

      // If placing queen in board[i][j] 

In [ ]:
examples = '''
### Example 1: Factorial calculation
### Instruction: Write factorial function in PascalABC.NET
### Response:
program FactorialExample;

function Factorial(n: integer): integer;
begin
  if n <= 1 then
    Result := 1
  else
    Result := n * Factorial(n - 1);
end;

begin
  WriteLn('Factorial of 5 is ', Factorial(5));
end.

### Example 2: Fibonacci sequence
### Instruction: Implement Fibonacci in PascalABC.NET
### Response:
program FibonacciExample;

function Fibonacci(n: integer): integer;
begin
  if n <= 1 then
    Result := n
  else
    Result := Fibonacci(n - 1) + Fibonacci(n - 2);
end;

begin
  for var i := 0 to 10 do
    WriteLn('Fib(', i, ') = ', Fibonacci(i));
end.
'''

target_request = '''
### Example 3: Tower of Hanoi
### Instruction: Implement Tower of Hanoi algorithm in PascalABC.NET
### Response:
'''
prompt = examples + target_request

messages=[
    { 'role': 'user', 'content': prompt}
]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
# tokenizer.eos_token_id is the id of <|EOT|> token
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


program HanoiExample;

procedure Hanoi(n: integer; source, target, auxiliary: integer);
begin
  if n > 0 then
  begin
    Hanoi(n - 1, source, auxiliary, target);
    WriteLn('Move disk ', n, ' from tower ', source, ' to tower ', target);
    Hanoi(n - 1, auxiliary, target, source);
  end;
end;

begin
  Hanoi(3, 1, 3, 2);
end.

This program will move the disks of a tower of 3 from tower 1 to tower 3 using tower 2 as the auxiliary peg. The procedure Hanoi is a recursive procedure that moves the n-1 disks from the source peg to the auxiliary peg, then moves the nth disk from the source peg to the target peg, and finally moves the n-1 disks from the auxiliary peg to the target peg.



In [ ]:
examples = '''
### Example 1: Array processing
### Instruction: Find maximum element in array
### Response:
program FindMaxExample;

function FindMax(arr: array of integer): integer;
begin
  Result := arr[0];
  for var i := 1 to High(arr) do
    if arr[i] > Result then
      Result := arr[i];
end;

begin
  var numbers := Arr(3, 7, 2, 9, 1);
  WriteLn('Maximum is ', FindMax(numbers));
end.

### Example 2: Backtracking pattern
### Instruction: Generate all permutations
### Response:
program PermutationsExample;

procedure GeneratePermutations(arr: array of integer; start: integer);
begin
  if start = Length(arr) then
  begin
    // Process permutation
    for var i := 0 to High(arr) do
      Write(arr[i], ' ');
    WriteLn;
    Exit;
  end;

  for var i := start to High(arr) do
  begin
    Swap(arr[start], arr[i]);
    GeneratePermutations(arr, start + 1);
    Swap(arr[start], arr[i]); // backtrack
  end;
end;

begin
  var elements := Arr(1, 2, 3);
  GeneratePermutations(elements, 0);
end.
'''

target_request = '''
### Example 3: N-Queens problem
### Instruction: Solve N-Queens problem in PascalABC.NET
### Response:
'''
prompt = examples + target_request

messages=[
    { 'role': 'user', 'content': prompt}
]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
# tokenizer.eos_token_id is the id of <|EOT|> token
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


The N-Queens problem is a classic problem in computer science and logic. It's a problem of placing N queens on an N×N chessboard such that no two queens threaten each other.

Here is a simple solution in PascalABC.NET:

```pascal
program NQueens;

function IsSafe(x, y: integer; var board: array of array of integer): boolean;
begin
  var i: integer;
  for i := 0 to y - 1 do
    if board[x][i] = 1 then
      Exit(false);
  for i := 0 to x - 1 do
    if board[i][y] = 1 then
      Exit(false);
  for i := 0 to min(x, y) - 1 do
    if board[x - i][y - i] = 1 then
      Exit(false);
  for i := 0 to min(x, y) - 1 do
    if board[x - i][y + i] = 1 then
      Exit(false);
  Result := true;
end;

procedure PlaceQueen(x, y: integer; var board: array of array of integer);
begin
  if x = Length(board) then
    Exit;
  if IsSafe(x, y, board) then
  begin
    board[x][y] := 1;
    if x = Length(board) - 1 then
      WriteLn('Solution:')
    else
      PlaceQueen(x + 1, 0, board);
    board[x][y] := 0;

In [6]:
!cd /content && python finetune_deepseekcoder2.py \
    --model_name_or_path deepseek-ai/deepseek-coder-1.3b-instruct \
    --data_path /content/train_data.json \
    --output_dir /content/drive/MyDrive/LoRa \
    --num_train_epochs 4 \
    --per_device_train_batch_size 8 \
    --gradient_accumulation_steps 2 \
    --save_strategy "steps" \
    --save_steps 25 \
    --save_total_limit 4 \
    --learning_rate 2e-4 \
    --warmup_steps 10 \
    --logging_steps 10 \
    --lr_scheduler_type "cosine" \
    --gradient_checkpointing True \
    --report_to "tensorboard" \
    --bf16 True \
    --model_max_length 1024

2025-11-04 12:30:52.973627: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762259452.993433    4496 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762259452.999578    4496 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762259453.015687    4496 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762259453.015715    4496 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762259453.015719    4496 computation_placer.cc:177] computation placer alr

In [ ]:
# Загружаем основную модель
model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"
model2 = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

# Загружаем твои LoRA адаптеры
model2 = PeftModel.from_pretrained(model2, "/content/drive/MyDrive/LoRa").to('cuda')

# Загружаем токенизатор
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Переводим в режим инференса
model2.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): ModulesToSaveWrapper(
          (original_module): Embedding(32256, 2048)
          (modules_to_save): ModuleDict(
            (default): Embedding(32256, 2048)
          )
        )
        (layers): ModuleList(
          (0-23): 24 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict(

In [ ]:
def test_model(model, prompt):
    # Форматируем как при обучении
    formatted_prompt = f'''
You are a PascalABC.NET coding assistant. Follow these rules:
1. Write only PascalABC.NET code
2. Provide complete programs when possible
3. Use modern PascalABC.NET features

### Instruction:
{prompt}
### Response:
'''

    inputs = tokenizer(formatted_prompt, return_tensors="pt", max_length=512, truncation=True).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1024,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

def test_model_raw(prompt):
  formatted_prompt = f'''
    ### Instruction:
    {prompt}
    ### Response:
    '''

  inputs = tokenizer(formatted_prompt, return_tensors="pt", max_length=512, truncation=True).to(model2.device)

  print(f"Inputs device: {inputs['input_ids'].device}")

  with torch.no_grad():
      outputs = model2.generate(
          **inputs,
          max_new_tokens=1024,
          temperature=0.7,
          do_sample=True,
          pad_token_id=tokenizer.eos_token_id
      )

  response = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return response


In [ ]:
# Тест 1: Простая функция
test_prompts = [
    """Напиши функцию для вычисления факториала числа""",

    """Напиши программу которая выводит все простые числа до 100""",

    """Создай класс "Студент" с полями: имя, возраст, средний балл""",

    """Напиши код для чтения данных из текстового файла и вывода содержимого""",

    """Реализуй алгоритм пузырьковой сортировки массива"""
]

# Запускаем тесты
print("🧪 ТЕСТИРУЕМ МОДЕЛЬ...\n")
for i, prompt in enumerate(test_prompts, 1):
    print(f"📝 Тест {i}: {prompt}")
    print("🔧 Ответ:")
    response = test_model(prompt)
    # Показываем только часть после Response:
    if "### Response:" in response:
        response = response.split("### Response:")[1].strip()
    print(response)
    print("\n" + "="*50 + "\n")

🧪 ТЕСТИРУЕМ МОДЕЛЬ...

📝 Тест 1: Напиши функцию для вычисления факториала числа
🔧 Ответ:
function Factorial(n: integer): integer;
begin
  if n<0 then
    Result := 1
  else
    Result := n * Factorial(n-1);
end;

begin
  var n := 20;
  WritelnFormat('Факториал {0} равен {1}',n,Factorial(n));
end.


📝 Тест 2: Напиши программу которая выводит все простые числа до 100
🔧 Ответ:
// Простые числа до 100
begin
  for var i:=2 to 100 do
  begin
    var f:=True;
    for var j:=2 to i.Sqrt.Trunc do
      if i.Divs(j) then
      begin
        f:=False;
        break;
      end;
    if f then
      Print(i);
  end;
end.


📝 Тест 3: Создай класс "Студент" с полями: имя, возраст, средний балл
🔧 Ответ:
type 
  Student = class
    name,surname: string;
    age,grade: integer;
  end;

begin
  var s := new Student('Иванов','Иван',20,87);
  s.Print;
end.


📝 Тест 4: Напиши код для чтения данных из текстового файла и вывода содержимого
🔧 Ответ:
var 
  f: Text;
  s: string;
begin
  assign(f,'a.txt');
  rese

In [ ]:
print(test_model("Выведи 2-у декартову степень множества символов, задаваемого строкой 'xyzw'"))

Inputs device: cuda:0

You are a PascalABC.NET coding assistant. Follow these rules:
1. Write only PascalABC.NET code
2. Provide complete programs when possible
3. Use modern PascalABC.NET features

### Instruction:
Выведи 2-у декартову степень множества символов, задаваемого строкой 'xyzw'
### Response:
begin
  var s := 'xyzw';
  var d := '';
  foreach var c in s do
    d += c + c;
  Println(d);
end.



In [ ]:
print(test_model("Выведи 2-у декартову степень множества символов, задаваемого строкой 'xyzw'"))


You are a PascalABC.NET coding assistant. Follow these rules:
1. Write only PascalABC.NET code
2. Provide complete programs when possible
3. Use modern PascalABC.NET features

### Instruction:
Выведи 2-у декартову степень множества символов, задаваемого строкой 'xyzw'
### Response:
begin
  var s := 'xyzw';
  s.ToD2.Println;
end.



In [ ]:
print(test_model("Print the Cartesian square of the character set from the string 'xyzw'"))

Inputs device: cuda:0

You are a PascalABC.NET coding assistant. Follow these rules:
1. Write only PascalABC.NET code
2. Provide complete programs when possible
3. Use modern PascalABC.NET features

### Instruction:
Print the Cartesian square of the character set from the string 'xyzw'
### Response:
uses GraphABC;

begin
  var s := 'xyzw';
  var max := Max(s);
  var r := 10.0;
  var dx := 2 * r;
  var dy := r;
  Rectangle(-r, -r, r*2+1, r+1);
  foreach var c in s do
  begin
    var x := r;
    var y := r;
    var q := True;
    case c of
      'x': (x,y) := (x+dx,y);
      'y': (x,y) := (x,y+dy);
      'z': (x,y) := (x+dx*5,y);
      'w': (x,y) := (x,y+dy*5);
    end;
    if q then
      Circle(x,y,r);
    TextOut(x,y,c);
  end;
  max += max*2;
  TextOut(max*2,0,'After');
  max += max*2;
  TextOut(0,max*2,'After');
end.



In [ ]:
print(test_model("Напиши код, решающий задачу Ханойских башен с визуализацией при помощи библиотеки GraphABC"))

Inputs device: cuda:0

You are a PascalABC.NET coding assistant. Follow these rules:
1. Write only PascalABC.NET code
2. Provide complete programs when possible
3. Use modern PascalABC.NET features

### Instruction:
Напиши код, решающий задачу Ханойских башен с визуализацией при помощи библиотеки GraphABC
### Response:
uses GraphABC;

procedure MoveFigure(n: integer; from, to, via: Rectangle);
begin
  if n > 1 then
    MoveFigure(n-1, via, to, from);
  via.MoveTo(to.Left, to.Top);
  via.LineTo(from.Left, from.Top);
  via.LineTo(from.Left, from.Top + 60);
  via.LineTo(from.Left + 60, from.Top + 60);
  via.LineTo(from.Left + 60, from.Top);
  via.LineTo(from.Left + 60, from.Top - 60);
  via.LineTo(from.Left + 60, from.Top - 60);
  via.LineTo(from.Left + 60, from.Top);
  via.LineTo(from.Left + 60, from.Top + 60);
  via.LineTo(from.Left + 60, from.Top + 60);
  via.LineTo(from.Left + 60, from.Top);
  via.LineTo(from.Left + 60, from.Top - 60);
  via.LineTo(from.Left + 60, from.Top - 60);
  vi

In [ ]:
print(test_model_raw("Implement Tower of Hanoi algorithm"))

Inputs device: cuda:0

    ### Instruction:
    Implement Tower of Hanoi algorithm
    ### Response:
    
    def hanoi(n, source, helper, target):
        if n > 0:
            hanoi(n - 1, source, target, helper)
            move(source, target)
            hanoi(n - 1, helper, source, target)
    
    def move(from_rod, to_rod):
        print('move disc from rod', from_rod, 'to rod', to_rod)

    n = 3
    hanoi(n, 'A', 'B', 'C')
    ### End: Implement Tower of Hanoi algorithm
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
    ###
   

In [ ]:
translate_prompt = f"""
Translate this Python code to PascalABC.NET:

Python code:
def hanoi(n, source, helper, target):
    if n > 0:
        hanoi(n - 1, source, target, helper)
        move(source, target)
        hanoi(n - 1, helper, source, target)

def move(from_rod, to_rod):
    print('move disc from rod', from_rod, 'to rod', to_rod)

n = 3
hanoi(n, 'A', 'B', 'C')

PascalABC.NET requirements:
- Use modern PascalABC.NET syntax
- Make it a complete program
- Do not change parameters order
"""

print(test_model_raw(translate_prompt))

Inputs device: cuda:0

    ### Instruction:
    
Translate this Python code to PascalABC.NET:

Python code:
def hanoi(n, source, helper, target):
    if n > 0:
        hanoi(n - 1, source, target, helper)
        move(source, target)
        hanoi(n - 1, helper, source, target)

def move(from_rod, to_rod):
    print('move disc from rod', from_rod, 'to rod', to_rod)

n = 3
hanoi(n, 'A', 'B', 'C')

PascalABC.NET requirements:
- Use modern PascalABC.NET syntax
- Make it a complete program
- Do not change parameters order

    ### Response:
    
procedure Move(from_rod, to_rod: string);
begin
  Write('move disc from rod ',from_rod,' to rod ',to_rod);
  NewLine;  
end;

procedure hanoi(n: integer; source, helper, target: string);
begin
  if n > 1 then
  begin
    hanoi(n-1, source, target, helper);
    Move(source, target);
    hanoi(n-1, helper, source, target);
  end;
end;

begin
  var n := 3;
  hanoi(n, 'A', 'B', 'C');
end.



In [ ]:
print(test_model_raw("Write an algorithm which solves n qweens problem in Python"))

Inputs device: cuda:0

    ### Instruction:
    Write an algorithm which solves n qweens problem in Python
    ### Response:
    
    def solve_n_queens(n):
        def can_place(queens, x, y):
            for i in range(len(queens)):
                if (queens[i] == y) or (queens[i] - x == y - queens[i]) or (queens[i] + x == y + queens[i]):
                    return False
            return True

        def place_queen(n, queens, x):
            for y in range(n):
                if can_place(queens, x, y):
                    if x == n-1:
                        print_board(n, queens)
                    else:
                        place_queen(n, queens + [y], x + 1)

        place_queen(n, [], 0)
    ### End of solution

    solve_n_queens(4)



In [ ]:
print(test_model("Напиши функцию быстрой сортировки"))


You are a PascalABC.NET coding assistant. Follow these rules:
1. Write only PascalABC.NET code
2. Provide complete programs when possible
3. Use modern PascalABC.NET features

### Instruction:
Напиши функцию быстрой сортировки
### Response:
function QSort(a: array of real): array of real;
begin
  if a.Length = 0 then
    exit(a);
  var pivot := a[0];
  var less := a.Where(y->y<pivot);
  var greater := a.Where(y->y>pivot);
  Result := QSort(less);
  Result += pivot;
  Result += QSort(greater);
end;

begin
  var a := ArrRandom(20);
  a.Println;
  a := QSort(a);
  a.Println;
end.



In [ ]:
import gc
del model
del model2
gc.collect()

975

In [ ]:
!cd /content && python finetune_deepseekcoder3.py \
    --model_name_or_path deepseek-ai/deepseek-coder-1.3b-instruct \
    --data_path /content/train_data.json \
    --output_dir /content/drive/MyDrive/LoRaDifferentRank \
    --num_train_epochs 5 \
    --per_device_train_batch_size 8 \
    --gradient_accumulation_steps 2 \
    --save_strategy "steps" \
    --save_steps 100 \
    --save_total_limit 3 \
    --learning_rate 2e-4 \
    --warmup_steps 10 \
    --logging_steps 10 \
    --lr_scheduler_type "cosine" \
    --gradient_checkpointing True \
    --report_to "tensorboard" \
    --bf16 True \
    --model_max_length 1024

2025-10-27 15:38:01.980042: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761579481.999616    1407 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761579482.005619    1407 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761579482.020374    1407 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761579482.020396    1407 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761579482.020400    1407 computation_placer.cc:177] computation placer alr

In [ ]:
model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"
model3 = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

model3 = PeftModel.from_pretrained(model3, "/content/drive/MyDrive/LoRaDifferentRank/syntax_adapter").to('cuda')

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model3.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): ModulesToSaveWrapper(
          (original_module): Embedding(32256, 2048)
          (modules_to_save): ModuleDict(
            (default): Embedding(32256, 2048)
          )
        )
        (layers): ModuleList(
          (0-23): 24 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict(

In [ ]:
print(test_model(model3, "Write a quick sort algorithm"))


You are a PascalABC.NET coding assistant. Follow these rules:
1. Write only PascalABC.NET code
2. Provide complete programs when possible
3. Use modern PascalABC.NET features

### Instruction:
Write a quick sort algorithm
### Response:
function quicksort(l: array of integer): array of integer;
begin
  if l.Length < 2 then
    exit(l);
  var pivot := l[0];
  var less := l.Where(x -> x <= pivot).ToArray;
  var greater := l.Where(x -> x > pivot).ToArray;
  Result := quicksort(less) + pivot + quicksort(greater);
end;

begin
  var a := ArrRandom(100);
  var b := quicksort(a);
  Println(b);
end.



In [ ]:
print(test_model(model3, "Write an algorithm to solve n-queens problem"))


You are a PascalABC.NET coding assistant. Follow these rules:
1. Write only PascalABC.NET code
2. Provide complete programs when possible
3. Use modern PascalABC.NET features

### Instruction:
Write an algorithm to solve n-queens problem
### Response:
function Ok(x: integer; y: integer): boolean;
begin
  Result := All(x:1..7) and All(y:1..7) and [(xx,yy) in Queens.Where(p->Abs(p.x-xx)=Abs(p.y-yy))]
end;

procedure WP(n: integer);
begin
  if n<=0 then
  begin
    Println('Solution numbers:');
    var x := 0;
    Queens.OrderBy(p->p.y).Println;
    Inc(x);
  end
  else
  begin
    for var y:=1 to 7 do
    begin
      if Ok(x,y) then
      begin
        var p := new PPoint(x,y);
        Queens.Add(p);
        if n=7 then
          Println('Found one solution');
        if n>7 then
        begin
          WP(n-1);
          if n=7 then
            Println('Found one solution');
        end
        else
        begin
          WP(n-1);
          if n=7 then
            Println('Found one s

In [ ]:
print(test_model(model3, "Write a Hanoi Towers algorithm"))


You are a PascalABC.NET coding assistant. Follow these rules:
1. Write only PascalABC.NET code
2. Provide complete programs when possible
3. Use modern PascalABC.NET features

### Instruction:
Write a Hanoi Towers algorithm
### Response:
procedure Towers(n: integer);
begin
  MoveTowers(1,3,n);
end;

procedure MoveDisk(frompeg,topeg: integer);
begin
  writelnFormat('Move disk: peg {0} to peg {1}',frompeg,topeg);
end;

procedure MoveTowers(frompeg,topeg,pegcount: integer);
begin
  if pegcount=1 then 
    MoveDisk(frompeg,topeg)
  else 
  begin
    MoveTowers(frompeg,pegcount-1,topeg);
    MoveDisk(frompeg,topeg);
    MoveTowers(pegcount-1,topeg,frompeg);
  end;
end;

begin
  var n: integer := 4;
  writeln('Start moving disks.');
  Towers(n);
  writeln('End moving disks.');
end.



In [ ]:
print(test_model(model3, '''Translate this python code to PascalABC.NET:
def hanoi(n, source, target, auxiliary):
    if n > 0:
        # Move n - 1 disks from source to auxiliary, so they are out of the way
        hanoi(n - 1, source, auxiliary, target)

        # Move the nth disk from source to target
        print('Move disk', n, 'from rod', source, 'to rod', target)

        # Move the n - 1 disks that we left on auxiliary to target
        hanoi(n - 1, auxiliary, target, source)

# Test the function
hanoi(3, 'A', 'C', 'B')
'''))


You are a PascalABC.NET coding assistant. Follow these rules:
1. Write only PascalABC.NET code
2. Provide complete programs when possible
3. Use modern PascalABC.NET features

### Instruction:
Translate this python code to PascalABC.NET:
def hanoi(n, source, target, auxiliary):
    if n > 0:
        # Move n - 1 disks from source to auxiliary, so they are out of the way
        hanoi(n - 1, source, auxiliary, target)

        # Move the nth disk from source to target
        print('Move disk', n, 'from rod', source, 'to rod', target)

        # Move the n - 1 disks that we left on auxiliary to target
        hanoi(n - 1, auxiliary, target, source)

# Test the function
hanoi(3, 'A', 'C', 'B')

### Response:
procedure hanoi(n: integer; source, target, auxiliary: char);
begin
  if n > 0 then
  begin
    hanoi(n - 1, source, auxiliary, target);
    writeln('Move disk ',n,' from rod ',source,' to rod ',target);
    hanoi(n - 1, auxiliary, target, source);
  end;
end;

begin
  hanoi(3, 'A',

In [ ]:
print(test_model(model3, '''Translate this python code to PascalABC.NET with minimal changes to the original solution:
  def solve_n_queens(n):
    def is_attack(i, j):
        # Check for any queen in the same row
        for k in range(0, n):
            if board[i][k] == 1:
                return True

        # Check for any queen in the same column
        for k in range(0, n):
            if board[k][j] == 1:
                return True

        # Check for any queen in the same diagonal
        for k in range(0, n):
            for l in range(0, n):
                if (k + l == i + j) or (k - l == i - j):
                    if board[k][l] == 1:
                        return True

        return False

    def solve_n_queens_util(n, i):
        # base case
        if i == n:
            return True

        for j in range(0, n):
            if not is_attack(i, j):
                # place queen at board[i][j]
                board[i][j] = 1

                # recur to place rest of the queens
                if solve_n_queens_util(n, i + 1) == True:
                    return True

                # If placing queen in board[i][j] doesn't lead to a solution, then
                # remove queen from board[i][j]
                board[i][j] = 0

        return False

    board = [[0]*n for _ in range(n)]

    if solve_n_queens_util(n, 0) == False:
        print("Solution does not exist")
        return False

    print_board(board, n)
    return True'''))


You are a PascalABC.NET coding assistant. Follow these rules:
1. Write only PascalABC.NET code
2. Provide complete programs when possible
3. Use modern PascalABC.NET features

### Instruction:
Translate this python code to PascalABC.NET with minimal changes to the original solution:
  def solve_n_queens(n):
    def is_attack(i, j):
        # Check for any queen in the same row
        for k in range(0, n):
            if board[i][k] == 1:
                return True

        # Check for any queen in the same column
        for k in range(0, n):
            if board[k][j] == 1:
                return True

        # Check for any queen in the same diagonal
        for k in range(0, n):
            for l in range(0, n):
                if (k + l == i + j) or (k - l == i - j):
                    if board[k][l] == 1:
                        return True

        return False

    def solve_n_queens_util(n, i):
        # base case
        if i == n:
            return True

        for j i

In [ ]:
print(test_model(model3, 'Write a function which checks whether string is a palindrom or not'))


You are a PascalABC.NET coding assistant. Follow these rules:
1. Write only PascalABC.NET code
2. Provide complete programs when possible
3. Use modern PascalABC.NET features

### Instruction:
Write a function which checks whether string is a palindrom or not
### Response:
function IsPalindrome(s: string): boolean;
begin
  Result := s = ReverseString(s);
end;

begin
  IsPalindrome('madam').Println;
  IsPalindrome('12321').Println;
end.



In [ ]:
print(test_model(model3, 'Write a function which checks whether string is a palindrom or not. Do not use standard PascalABC.NET functions.'))


You are a PascalABC.NET coding assistant. Follow these rules:
1. Write only PascalABC.NET code
2. Provide complete programs when possible
3. Use modern PascalABC.NET features

### Instruction:
Write a function which checks whether string is a palindrom or not. Do not use standard PascalABC.NET functions.
### Response:
function Is_Palindrome(s: string): boolean;
begin
  var L := s.Length;
  var k := 0;
  while k < L div 2 do
  begin
    if s[k] <> s[L-k-1] then
      exit(False);
    k += 1;
  end;
  Result := True;
end;

begin
  var s := 'madam';
  Is_Palindrome(s).Println;
end.



In [ ]:
print(test_model(model3, '''
Implement depth-first search for a graph represented as an adjacency list. The code should:

- Take a starting node and a graph structure as input
- Return the order in which nodes are visited
- Handle cycles using a visited set/map
- Work for both connected and disconnected graphs

Include both recursive and iterative implementations using a stack.'''))


You are a PascalABC.NET coding assistant. Follow these rules:
1. Write only PascalABC.NET code
2. Provide complete programs when possible
3. Use modern PascalABC.NET features

### Instruction:

Implement depth-first search for a graph represented as an adjacency list. The code should:

- Take a starting node and a graph structure as input
- Return the order in which nodes are visited
- Handle cycles using a visited set/map
- Work for both connected and disconnected graphs

Include both recursive and iterative implementations using a stack.
### Response:
// Depth-first search for a graph represented as an adjacency list
procedure DFS(start: integer; graph: array of array of integer; visited: array of boolean);
// recursive implementation
begin
  Print(start);
  visited[start] := True;
  foreach var x in graph[start] do
    if not visited[x] then
      DFS(x, graph, visited);
end;

procedure DFS(start: integer; graph: array of array of integer);
// iterative implementation
// uses stack

In [ ]:
print(test_model(model3, '''
Write a comprehensive binary search implementation that includes:

1. Standard binary search to find if a target exists
2. Find the first occurrence of a target in a sorted array with duplicates
3. Find the insertion position for a target in a sorted array
4. Handle edge cases like empty arrays and targets not found

Each variation should be implemented as a separate function.'''))


You are a PascalABC.NET coding assistant. Follow these rules:
1. Write only PascalABC.NET code
2. Provide complete programs when possible
3. Use modern PascalABC.NET features

### Instruction:

Write a comprehensive binary search implementation that includes:

1. Standard binary search to find if a target exists
2. Find the first occurrence of a target in a sorted array with duplicates
3. Find the insertion position for a target in a sorted array
4. Handle edge cases like empty arrays and targets not found

Each variation should be implemented as a separate function.
### Response:
function BinarySearch(a: array of integer; x: integer): boolean;
begin
  var lo := 0;
  var hi := a.Length - 1;
  while lo <= hi do
  begin
    var mi := (lo + hi) div 2;
    if a[mi] < x then lo := mi + 1
    else if a[mi] > x then hi := mi - 1
    else exit True;
  end;
  Result := False;
end;

function FirstIndex(a: array of integer; x: integer): integer;
begin
  var lo := 0;
  var hi := a.Length - 1;
  w

In [ ]:
print(test_model(model3, '''
Write a program to find all prime numbers up to a given number N using the Sieve of Eratosthenes algorithm. Do not declare variables outside of begin-end block.'''))


You are a PascalABC.NET coding assistant. Follow these rules:
1. Write only PascalABC.NET code
2. Provide complete programs when possible
3. Use modern PascalABC.NET features

### Instruction:

Write a program to find all prime numbers up to a given number N using the Sieve of Eratosthenes algorithm. Do not declare variables outside of begin-end block.
### Response:
begin
  var N:=ReadInt('Enter N: ');
  var a:=ArrGen(N+1,i->True);
  a[0:2] := (0,0);
  for var i:=2 to Round(Sqrt(N)) do
    if a[i] then
      for var j:=i*i to N step i do
        a[j] := False;
  Println('Prime numbers: ',Where(a));
end.



In [ ]:
import transformers
import json
import numpy as np

tokenizer = transformers.AutoTokenizer.from_pretrained(
        "deepseek-ai/deepseek-coder-1.3b-instruct",
        model_max_length=1024,
        padding_side="right",
        use_fast=True,
        trust_remote_code=True
    )


with open('/content/train_data.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)

text_lengths = []
for item in train_data:
    full_text = item['instruction'] + " " + item['output']
    tokens_length = len(tokenizer.encode(full_text))

    if tokens_length > 1024:
      print(item['instruction'], tokens_length)
    text_lengths.append(tokens_length)

# Статистика
print(f"Всего примеров: {len(text_lengths)}")
print(f"Мин. длина: {np.min(text_lengths)}")
print(f"Макс. длина: {np.max(text_lengths)}")
print(f"Средняя длина: {np.mean(text_lengths):.1f}")
print(f"Медиана: {np.median(text_lengths)}")
print(f"95-й перцентиль: {np.percentile(text_lengths, 95)}")
print(f"99-й перцентиль: {np.percentile(text_lengths, 99)}")

Token indices sequence length is longer than the specified maximum sequence length for this model (1633 > 1024). Running this sequence through the model will result in indexing errors


Implement the classic 15 puzzle game in PascalABC.NET using GraphABC and ABCObjects with tile movement, shuffling, and win condition detection 1633
Create interactive standard colors palette with ABCObjects using reflection 1365
Create interactive standard colors palette with WPFObjects using reflection 1385
Implement Tower of Hanoi algorithm with graphical visualization in GraphABC 1150
Create a shooting game in PascalABC.NET using ABCObjects where the player controls a rectangle at the bottom and shoots bullets at colored enemy rectangles. Implement keyboard controls, collision detection, scoring system, and game states. 1796
Implement Conway's Game of Life on a toroidal grid in PascalABC.NET using GraphWPF with optimization through uniform grid hashing. The program should handle cell updates efficiently and display generation information. 2621
Implement complex fish ecosystem simulation with predator-prey behavior using GraphABC in PascalABC.NET 3676
Implement word game with letter 